In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords

In [60]:
df = pd.read_csv("IMDB/valid.csv")
df

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
...,...,...
4995,"Man, I loved this movie! This really takes me ...",1
4996,Recovery is an incredibly moving piece of work...,1
4997,"You can take the crook out of the joint, but i...",1
4998,FUTZ is the only show preserved from the exper...,1


In [61]:
stop_words = stopwords.words("english")
# wordnet = WordNetLemmatizer()

In [62]:
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [63]:
df['clean_text'] = df.text.apply(text_preproc)

In [64]:
df

,text,label,clean_text
0,It's been about 14 years since Sharon Stone aw...,0,years since sharon stone awarded viewers leg ...
1,someone needed to make a car payment... this i...,0,someone needed make car payment truly awful ma...
2,The Guidelines state that a comment must conta...,0,guidelines state comment must contain minimum ...
3,This movie is a muddled mish-mash of clichés f...,0,movie muddled mish mash clichs recent cinema p...
4,Before Stan Laurel became the smaller half of ...,0,stan laurel became smaller half all time great...
...,...,...,...
4995,"Man, I loved this movie! This really takes me ...",1,man loved movie really takes back kid days tea...
4996,Recovery is an incredibly moving piece of work...,1,recovery incredibly moving piece work handling...
4997,"You can take the crook out of the joint, but i...",1,take crook joint seems exceedingly difficult t...
4998,FUTZ is the only show preserved from the exper...,1,futz show preserved experimental theatre movem...


In [65]:
df.drop(columns=["text"], inplace= True)
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
df.to_csv("IMDB/train.csv")

NameError: name 'df' is not defined

In [16]:
import tensorflow as tf

In [3]:
df1 = pd.read_csv("IMDB/train.csv")
df1

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [4]:
x = [len(word_tokenize(sent)) for sent in df1["text"]]

NameError: name 'word_tokenize' is not defined

In [143]:
np.mean(x)

129.3992

In [10]:
from tensorflow.keras.preprocessing.text import one_hot

In [11]:
df = pd.read_csv("IMDB/train.csv")

In [12]:
voc_size=10000

In [13]:
onehot_repr = []

In [14]:
onehot_repr = []
onehot_repr=onehot_repr+[one_hot(sents,voc_size)for sents in df["text"][:40000]] 
print(len(onehot_repr))

40000


In [15]:
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [16]:
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[ 535 9678 9515 ... 4063 8168 8293]
 [7581 8686 2861 ... 3716 1002 9442]
 [6538 7536  700 ... 7389 6531 1745]
 ...
 [ 785 9573 4604 ... 8232 4311 8592]
 [7536 9678 6696 ... 8686  362 3215]
 [5277 9678 4750 ... 4399 2279 6231]]


In [17]:
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')


In [18]:
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,366,593
Trainable params: 1,366,593
Non-trainable params: 0
_________________________________________________________________


In [20]:
train_x = embedded_docs
train_y = df.label
train_x.shape, train_y.shape

((40000, 100), (40000,))

In [22]:
val = pd.read_csv("IMDB/valid.csv")
val

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
...,...,...
4995,"Man, I loved this movie! This really takes me ...",1
4996,Recovery is an incredibly moving piece of work...,1
4997,"You can take the crook out of the joint, but i...",1
4998,FUTZ is the only show preserved from the exper...,1


In [23]:
onehot_reprv = []

In [24]:
onehot_reprv=onehot_reprv+[one_hot(sents,voc_size)for sents in df["text"][:5000]] 
print(len(onehot_reprv))

5000


In [25]:

val_x=pad_sequences(onehot_reprv,padding='pre',maxlen=sent_length)
val_y = val.label
print(val_x.shape,val_y.shape)

(5000, 100) (5000,)


In [26]:
model.fit(train_x, train_y, batch_size=32, 
          epochs=3,validation_data=(val_x,val_y))

Train on 40000 samples, validate on 5000 samples
Epoch 1/3
40000/40000 [==============================] - 696s 17ms/sample - loss: 0.4352 - accuracy: 0.7964 - val_loss: 1.3390 - val_accuracy: 0.5012
Epoch 2/3
40000/40000 [==============================] - 672s 17ms/sample - loss: 0.3036 - accuracy: 0.8722 - val_loss: 1.4820 - val_accuracy: 0.4974
Epoch 3/3
 2112/40000 [>.............................] - ETA: 11:07 - loss: 0.1959 - accuracy: 0.9308

KeyboardInterrupt: 

In [87]:
test = pd.read_csv("IMDB/test.csv")
test

,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
...,...,...
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0


In [88]:
onehot_reprt = []
s = []
for x in test["text"]:
    s.append("".join(x.split("<br />")))
len(s)

5000

In [89]:
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in s] 
print(len(onehot_reprt))

5000


In [90]:

test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
test_y = test.label
print(test_x.shape,test_y.shape)

(5000, 100) (5000,)


In [91]:
model.evaluate(test_x, test_y, batch_size=32)

5000/5000 [==============================] - 9s 2ms/sample - loss: 0.4464 - accuracy: 0.8528


[0.4463780487537384, 0.8528]

In [92]:
a = model.predict_classes(test_x)
a

array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [98]:
a[2][0]
len(a)

5000

In [66]:
onehot_reprt = []
sentence = [s]
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in s] 
test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)

In [67]:
a = model.predict_classes(test_x)
a

array([[1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1]])

In [179]:
a[0][0]

0

In [178]:
test_y


0       0
1       0
2       0
3       1
4       0
       ..
4995    1
4996    1
4997    0
4998    0
4999    0
Name: label, Length: 5000, dtype: int64

In [99]:
count = 0
for x in range(5000):
    if a[x][0] != test_y[x]:
        count+=1
        print(x+2)
count

2
3
5
18
28
35
40
61
70
81
93
98
99
100
102
104
111
114
128
132
134
138
139
145
147
160
165
169
173
174
180
183
185
205
207
209
212
213
217
221
224
232
235
241
246
250
251
256
261
263
266
268
272
277
278
296
309
313
321
325
333
340
348
351
359
361
363
370
373
377
379
381
383
386
419
422
432
438
448
451
460
477
480
485
490
494
499
506
509
513
515
517
527
532
534
540
542
553
571
580
587
589
592
594
595
601
609
615
631
636
641
642
651
661
663
665
675
685
688
691
693
707
709
710
716
729
736
740
750
756
759
762
785
792
810
811
817
844
848
853
857
867
871
895
901
902
913
922
939
954
960
961
963
970
994
997
1000
1006
1008
1017
1019
1021
1042
1043
1046
1049
1050
1056
1058
1076
1077
1080
1103
1117
1120
1126
1127
1132
1138
1143
1148
1162
1163
1170
1178
1179
1182
1189
1195
1202
1205
1208
1209
1217
1240
1243
1245
1251
1256
1271
1284
1288
1297
1305
1307
1309
1318
1342
1354
1370
1388
1399
1413
1431
1433
1438
1441
1442
1456
1462
1473
1477
1478
1487
1488
1499
1510
1511
1515
1518
1530
1532
1538
1545
15

736

In [140]:
s = "I always wrote this series off as being a complete stink-fest because Jim Belushi was involved in it, and heavily. But then one day a tragic happenstance occurred. After a White Sox game ended I realized that the remote was all the way on the other side of the room somehow. Now I could have just gotten up and walked across the room to get the remote, or even to the TV to turn the channel. But then why not just get up and walk across the country to watch TV in another state? Nuts to that, I said. So I decided to just hang tight on the couch and take whatever Fate had in store for me. What Fate had in store was an episode of this show, an episode about which I remember very little except that I had once again made a very broad, general sweeping blanket judgment based on zero objective or experiential evidence with nothing whatsoever to back my opinions up with, and once again I was completely right! This show is a total crud-pie! Belushi has all the comedic delivery of a hairy lighthouse foghorn. The women are physically attractive but too Stepford-is to elicit any real feeling from the viewer. There is absolutely no reason to stop yourself from running down to the local TV station with a can of gasoline and a flamethrower and sending every copy of this mutt howling back to hell. <br /><br />Except.. <br /><br />Except for the wonderful comic sty lings of Larry Joe Campbell, America's Greatest Comic Character Actor. This guy plays Belushi's brother-in-law, Andy, and he is gold. How good is he really? Well, aside from being funny, his job is to make Belushi look good. That's like trying to make butt warts look good. But Campbell pulls it off with style. Someone should invent a Nobel Prize in Comic Buffoonery so he can win it every year. Without Larry Joe this show would consist of a slightly vacant looking Courtney Thorne-Smith smacking Belushi over the head with a frying pan while he alternately beats his chest and plays with the straw on the floor of his cage. 5 stars for Larry Joe Campbell designated Comedic Bacon because he improves the flavor of everything he's in!"

In [141]:
s = "".join(s.split("<br />"))
s = "I always wrote this series off as being a complete stink-fest because Jim Belushi was involved in it, and heavily. But then one day a tragic happenstance occurred. After a White Sox game ended I realized that the remote was all the way on the other side of the room somehow. Now I could have just gotten up and walked across the room to get the remote, or even to the TV to turn the channel. But then why not just get up and walk across the country to watch TV in another state? Nuts to that, I said. So I decided to just hang tight on the couch and take whatever Fate had in store for me. What Fate had in store was an episode of this show, an episode about which I remember very little except that I had once again made a very broad, general sweeping blanket judgment based on zero objective or experiential evidence with nothing whatsoever to back my opinions up with, and once again I was completely right! This show is a total crud-pie! Belushi has all the comedic delivery of a hairy lighthouse foghorn. The women are physically attractive but too Stepford-is to elicit any real feeling from the viewer. There is absolutely no reason to stop yourself from running down to the local TV station with a can of gasoline and a flamethrower and sending every copy of this mutt howling back to hell. <br /><br />Except.. <br /><br />Except for the wonderful comic sty lings of Larry Joe Campbell, America's Greatest Comic Character Actor. This guy plays Belushi's brother-in-law, Andy, and he is gold. How good is he really? Well, aside from being funny, his job is to make Belushi look good. That's like trying to make butt warts look good. But Campbell pulls it off with style. Someone should invent a Nobel Prize in Comic Buffoonery so he can win it every year. Without Larry Joe this show would consist of a slightly vacant looking Courtney Thorne-Smith smacking Belushi over the head with a frying pan while he alternately beats his chest and plays with the straw on the floor of his cage. 5 stars for Larry Joe Campbell designated Comedic Bacon because he improves the flavor of everything he's in!"
s = [s]
# s = s.split(".")
# len(s)

In [142]:
onehot_reprt = []
s = "I always wrote this series off as being a complete stink-fest because Jim Belushi was involved in it, and heavily. But then one day a tragic happenstance occurred. After a White Sox game ended I realized that the remote was all the way on the other side of the room somehow. Now I could have just gotten up and walked across the room to get the remote, or even to the TV to turn the channel. But then why not just get up and walk across the country to watch TV in another state? Nuts to that, I said. So I decided to just hang tight on the couch and take whatever Fate had in store for me. What Fate had in store was an episode of this show, an episode about which I remember very little except that I had once again made a very broad, general sweeping blanket judgment based on zero objective or experiential evidence with nothing whatsoever to back my opinions up with, and once again I was completely right! This show is a total crud-pie! Belushi has all the comedic delivery of a hairy lighthouse foghorn. The women are physically attractive but too Stepford-is to elicit any real feeling from the viewer. There is absolutely no reason to stop yourself from running down to the local TV station with a can of gasoline and a flamethrower and sending every copy of this mutt howling back to hell. <br /><br />Except.. <br /><br />Except for the wonderful comic sty lings of Larry Joe Campbell, America's Greatest Comic Character Actor. This guy plays Belushi's brother-in-law, Andy, and he is gold. How good is he really? Well, aside from being funny, his job is to make Belushi look good. That's like trying to make butt warts look good. But Campbell pulls it off with style. Someone should invent a Nobel Prize in Comic Buffoonery so he can win it every year. Without Larry Joe this show would consist of a slightly vacant looking Courtney Thorne-Smith smacking Belushi over the head with a frying pan while he alternately beats his chest and plays with the straw on the floor of his cage. 5 stars for Larry Joe Campbell designated Comedic Bacon because he improves the flavor of everything he's in!"
s = [s]
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in s] 
test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
a = model.predict_classes(test_x)
a

array([[1]])

In [128]:
np.count_nonzero(a == 1)

6

In [129]:
len(a)

8

In [132]:
l_out = []
for s in test["text"]:
    s = "".join(s.split("<br />"))
    s = s.split(".")
    onehot_reprt = []
    onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in s] 
    test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
    a = model.predict_classes(test_x)
    l_out.append(1 if np.count_nonzero(a == 1) > len(a)/2 else 0)
len(l_out)

5000

In [134]:
count = 0
for x in range(5000):
    if l_out[x] != test_y[x]:
        count+=1
        print(x+2)
count

2
3
6
8
12
14
16
17
19
26
30
36
38
40
48
54
56
57
63
64
69
70
72
74
76
77
79
82
85
86
89
90
91
92
94
95
97
100
102
105
108
109
110
111
112
113
114
117
118
119
120
123
126
129
130
131
132
133
134
139
140
143
144
145
146
150
151
154
159
160
161
164
165
166
167
169
170
175
179
181
183
184
186
187
191
193
194
195
197
200
202
205
208
209
216
221
222
224
225
226
227
232
237
241
242
249
254
260
262
263
266
268
269
274
278
279
281
283
284
285
289
290
291
296
297
298
301
302
303
304
306
308
309
316
320
321
322
324
326
328
332
336
337
338
339
342
344
349
351
352
354
355
358
359
361
363
364
365
367
370
374
378
379
380
381
382
383
388
389
391
392
393
394
395
402
406
407
410
411
413
416
419
420
421
426
429
432
433
434
437
440
442
444
446
448
450
451
453
454
460
464
465
466
468
469
471
472
473
475
476
477
479
483
486
490
494
500
502
508
509
510
515
516
517
518
521
522
525
528
530
531
534
540
545
547
548
551
552
553
554
562
571
573
574
579
580
581
583
585
590
592
593
594
595
597
599
601
602
607
608
6

2010

In [ ]:
df = pd.read_

In [1]:
import pandas as pd

In [3]:
df = pd.read_json("books/books_5.json")

MemoryError: 

In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('C:/Users/Adarsh Kumar/Downloads/books_5.json.gz')
df

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = "".join(x.split("<br />"))
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [3]:
train = pd.read_csv("IMDB/train.csv")
valid = pd.read_csv("IMDB/valid.csv")
test = pd.read_csv("IMDB/test.csv")

In [4]:
train['clean_text'] = train.text.apply(text_preproc)
valid['clean_text'] = valid.text.apply(text_preproc)
test['clean_text'] = test.text.apply(text_preproc)
train.drop(columns=["text"], inplace= True)
valid.drop(columns=["text"], inplace= True)
test.drop(columns=["text"], inplace= True)

In [5]:
voc_size = 10000
onehot_repr = []
onehot_repr=onehot_repr+[one_hot(sents,voc_size)for sents in train["clean_text"][:40000]] 
print(len(onehot_repr))
onehot_reprv = []
onehot_reprv=onehot_reprv+[one_hot(sents,voc_size)for sents in valid["clean_text"][:5000]] 
print(len(onehot_reprv))
onehot_reprt = []
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in test["clean_text"][:5000]] 
print(len(onehot_reprt))

40000
5000
5000


In [6]:
sent_length=100
train_x=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
train_y = train.label
print(train_x)
print(train_x.shape, train_y.shape)

val_x=pad_sequences(onehot_reprv,padding='pre',maxlen=sent_length)
val_y = valid.label
print(val_x)
print(val_x.shape, val_y.shape)

test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
test_y = test.label
print(test_x)
print(test_x.shape, test_y.shape)

[[   0    0    0 ... 8870 9069 2032]
 [1827 1134 9948 ... 2432 8919 2574]
 [4349 4158  144 ... 6506 8161 8389]
 ...
 [ 229 1205 2968 ... 5519 5189 3751]
 [   0    0    0 ... 9729 7306   27]
 [   0    0    0 ... 6903 6132 8203]]
(40000, 100) (40000,)
[[1487 2416 2516 ... 8870 5069 9958]
 [   0    0    0 ... 3793 1514 8393]
 [   0    0    0 ... 5542 5381 8161]
 ...
 [   0    0    0 ... 1478  380 3693]
 [7268 3711 1741 ... 2905 6629 4299]
 [   0    0    0 ... 8788 8256 8694]]
(5000, 100) (5000,)
[[8788 2574 2094 ... 4643 5396 5053]
 [   0    0 1480 ... 6629 7622 5922]
 [3793 4354 2714 ... 3793 5135 6052]
 ...
 [   0    0    0 ... 1644 3666 3482]
 [   0    0    0 ... 9268 9268 9268]
 [   0    0    0 ... 2909 5230 7316]]
(5000, 100) (5000,)


In [7]:
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,366,593
Trainable params: 1,366,593
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(train_x, train_y, batch_size=32, 
          epochs=1,validation_data=(val_x,val_y))

Train on 40000 samples, validate on 5000 samples
40000/40000 [==============================] - 639s 16ms/sample - loss: 0.4212 - accuracy: 0.8091 - val_loss: 0.3795 - val_accuracy: 0.8356


In [9]:
model.evaluate(test_x, test_y, batch_size=32)

5000/5000 [==============================] - 8s 2ms/sample - loss: 0.3805 - accuracy: 0.8342


[0.3805092618703842, 0.8342]

In [25]:
onehot_reprx = []
s = "i can't believe people are looking for a plot in this film"
s = [s]
onehot_reprx=onehot_reprx+[one_hot(sents,voc_size)for sents in s] 
testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
testing_x.shape
# model.predict_classes(testing_x)

(1, 100)

In [13]:
import tensorflow as tf
tf.keras.models.save_model(model,"model.h5")

In [18]:
loaded_model = tf.keras.models.load_model('model.h5')

In [26]:
loaded_model.predict(testing_x)

array([[0.11740264]], dtype=float32)